In [1]:
#import pyspark
#from pyspark.sql import SQLContext
#from pyspark.sql.functions import year,month,hour, when, col, date_format, to_timestamp, round, coalesce
#from pyspark.sql import SparkSession
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import numpy as np
from sklearn.ensemble import RandomForestRegressor

In [2]:
#sc = pyspark.SparkSession('local', 'test')
#sc = pyspark.SparkContext(appName="HW3-Q1")
#sqlContext = SQLContext(sc)

In [3]:
df = pd.read_csv('pollution_us_2000_2016.csv')

In [4]:
df["Date"] = pd.to_datetime(df["Date Local"])
df['year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month

In [5]:
#df=df.withColumn("year",year("Date Local"))
df.head()

,Unnamed: 0,State Code,County Code,Site Num,Address,State,County,City,Date Local,NO2 Units,...,SO2 1st Max Hour,SO2 AQI,CO Units,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI,Date,year,Month
0,0,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,21,13.0,Parts per million,1.145833,4.2,21,NaN,2000-01-01,2000,1
1,1,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,21,13.0,Parts per million,0.878947,2.2,23,25.0,2000-01-01,2000,1
2,2,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,23,NaN,Parts per million,1.145833,4.2,21,NaN,2000-01-01,2000,1
3,3,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,23,NaN,Parts per million,0.878947,2.2,23,25.0,2000-01-01,2000,1
4,4,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-02,Parts per billion,...,22,4.0,Parts per million,0.850000,1.6,23,NaN,2000-01-02,2000,1


In [6]:
df1=df[["State","County","year","NO2 Mean","CO Mean",'O3 Mean','SO2 Mean']]
df1 = df1.groupby(["State","County","year"]).mean(["NO2 Mean","CO Mean",'O3 Mean','SO2 Mean'])

In [7]:
#df1=df[["State","County","year","NO2 Mean","CO Mean",'O3 Mean','SO2 Mean']]\
#.groupby("State","County","year").avg("NO2 Mean","CO Mean",'O3 Mean','SO2 Mean')

In [8]:
df1p=df1.copy()

In [9]:
df2p=pd.read_csv("death_data.csv")

In [10]:
us_state_abbrev={'AK':'Alaska','AL':'Alabama','AR':'Arkansas','AZ':'Arizona','CA': 'California','CO':'Colorado',\
                                 'CT':'Connecticut','DE':'Delaware','FL':'Florida','GA':'Georgia','HI':'Hawaii','IA':'Iowa',\
                                 'ID':'Idaho','IL': 'Illinois','IN':'Indiana','KS':'Kansas','KY':'Kentucky','LA':'Louisiana', \
                                'MA':'Massachusetts','MD':'Maryland','ME':'Maine','MI':'Michigan','MN':'Minnesota','MO':'Missouri', \
                                'NC':'North Carolina','ND':'North Dakota','NH':'New Hampshire','NJ':'New Jersey','NM':'New Mexico', \
                                'NV':'Nevada','NY':'New York','OH':'Ohio','OK':'Oklahoma','OR':'Oregon','PA':'Pennsylvania' ,\
                                'RI':'Rhode Island','SC':'South Carolina','SD':'South Dakota','TN':'Tennessee','TX':'Texas', \
                                'UT':'Utah','VA':'Virginia','WA':'Washington','WI':'Wisconsin','WY':'Wyoming'}

In [11]:
df2p['State']=df2p['State'].map(us_state_abbrev)

In [12]:
df_all=df1p.sort_values(by=['year'])

In [13]:
df3p=pd.merge(df_all,df2p,how="inner", left_on=['State','County','year'], right_on=['State','County_Name','Year'])

In [14]:
df3p=df3p.T.drop_duplicates().T

In [15]:
df3p['County'] = df3p['County_Name']
df3p['year'] = df3p['Year']

In [16]:
df4p=pd.read_csv("manufaturing_totals_2000_to_2016_restructure.csv")

In [17]:
df5p=df4p.dropna()

In [18]:
#df5p=df4p.melt(id_vars=["county", "state"], 
#        var_name="year", 
#        value_name="number of factories")
df5p

,county,state,year,fat_amount
0,Los Angeles,CA,2000,35154
1,Cook,IL,2000,14998
2,Orange,CA,2000,11332
3,Harris,TX,2000,8502
4,New York,NY,2000,8382
...,...,...,...,...
53222,Mcmullen,TX,2016,5
53223,Leslie,KY,2016,5
53224,Lyman,SD,2016,5
53225,Loup,NE,2016,5


In [19]:
df5p['state']=df5p['state'].map(us_state_abbrev)

In [20]:
def swap_columns(df, col1, col2):
    col_list = list(df.columns)
    x, y = col_list.index(col1), col_list.index(col2)
    col_list[y], col_list[x] = col_list[x], col_list[y]
    df = df[col_list]
    return df

In [21]:
df5p = swap_columns(df5p, 'county', 'state')
df5p=df5p.rename(columns={"state": "State", "county": "County", "fat_amount": "num_factory"})

In [22]:
df5p

,State,County,year,num_factory
0,California,Los Angeles,2000,35154
1,Illinois,Cook,2000,14998
2,California,Orange,2000,11332
3,Texas,Harris,2000,8502
4,New York,New York,2000,8382
...,...,...,...,...
53222,Texas,Mcmullen,2016,5
53223,Kentucky,Leslie,2016,5
53224,South Dakota,Lyman,2016,5
53225,NaN,Loup,2016,5


In [23]:
df6p=pd.merge(df3p,df5p,how="inner", on=['State','County','year'])

In [24]:
df6p

,State,NO2 Mean,CO Mean,O3 Mean,SO2 Mean,Year,County_Name,Population,Area,Deaths,...,Crude Rate,pollution_Crude_Rate,resp_Crude_Rate,circ_Crude_Rate,canc_Crude_Rate,resp_circ_Crude_Rate,population_density,County,year,num_factory
0,California,21.793608,0.843789,0.023372,1.173213,2000,Riverside,1545387,7303.81,12041,...,779.157583,592.66708,81.597684,329.820297,181.249098,411.417981,211.586419,Riverside,2000,2800
1,Texas,16.867261,0.382191,0.019523,6.106436,2000,Harris,3400578,1777.89,19521,...,574.04947,392.609727,40.610743,217.080743,134.91824,257.691487,1912.704386,Harris,2000,8502
2,Florida,12.453038,0.725072,0.026261,1.973912,2000,Orange,896344,1004.31,6188,...,690.359951,485.751006,65.153557,257.378863,163.218586,322.532421,892.497336,Orange,2000,1728
3,California,20.298693,0.712394,0.016592,2.352182,2000,San Francisco,776733,231.89,6363,...,819.20042,604.96979,85.74375,322.762133,196.463907,408.505883,3349.57523,San Francisco,2000,2100
4,Michigan,22.640061,0.277001,0.02335,5.415251,2000,Wayne,2061162,672.26,20218,...,980.903005,702.467831,74.957718,412.340224,215.16989,487.297942,3066.0191,Wayne,2000,4336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1089,California,2.014554,0.265178,0.027173,0.364641,2016,Humboldt,136646,4052.52,1196,...,875.254307,543.740761,68.059072,286.140831,189.540857,354.199903,33.718773,Humboldt,2016,640
1090,Indiana,13.857478,0.244739,0.026542,0.392973,2016,Marion,941229,403.11,8178,...,868.864007,503.384405,96.894592,224.918697,181.571116,321.813289,2334.918509,Marion,2016,4290
1091,Pennsylvania,5.870183,0.127532,0.030331,0.624739,2016,Erie,276207,1558.49,2782,...,1007.215603,625.255696,115.855138,286.379418,223.02114,402.234556,177.227316,Erie,2016,2320
1092,Arkansas,8.661113,0.322073,0.02601,0.6638,2016,Pulaski,393250,807.91,3517,...,894.342022,545.200254,67.387158,294.469167,183.343929,361.856325,486.749762,Pulaski,2016,1425


In [25]:
df6p.columns

Index(['State', 'NO2 Mean', 'CO Mean', 'O3 Mean', 'SO2 Mean', 'Year',
       'County_Name', 'Population', 'Area', 'Deaths', 'pollution_death',
       'resp_death', 'circ_death', 'canc_death', 'resp_circ_death',
       'Crude Rate', 'pollution_Crude_Rate', 'resp_Crude_Rate',
       'circ_Crude_Rate', 'canc_Crude_Rate', 'resp_circ_Crude_Rate',
       'population_density', 'County', 'year', 'num_factory'],
      dtype='object')

In [26]:
df7p=df6p[['NO2 Mean','CO Mean','O3 Mean','SO2 Mean', 'population_density','num_factory']]
df7p.head()

,NO2 Mean,CO Mean,O3 Mean,SO2 Mean,population_density,num_factory
0,21.793608,0.843789,0.023372,1.173213,211.586419,2800
1,16.867261,0.382191,0.019523,6.106436,1912.704386,8502
2,12.453038,0.725072,0.026261,1.973912,892.497336,1728
3,20.298693,0.712394,0.016592,2.352182,3349.57523,2100
4,22.640061,0.277001,0.02335,5.415251,3066.0191,4336


In [27]:
cars=pd.read_csv("CA_county_cars.csv")

In [28]:
df7p_cars=pd.merge(df6p,cars,how="inner", on=['State','County','year'])

In [29]:
df8p=df7p_cars[['NO2 Mean','CO Mean','O3 Mean','SO2 Mean', 'population_density','num_factory','Autos','Trucks','Total Vehicles']]

In [30]:
df8p = df8p.astype(float)
df7p = df7p.astype(float)

In [31]:
#df7p.to_csv('df7.csv')
df7p.corr()

,NO2 Mean,CO Mean,O3 Mean,SO2 Mean,population_density,num_factory
NO2 Mean,1.000000,0.587601,-0.525557,0.416436,0.437043,0.294672
CO Mean,0.587601,1.000000,-0.392298,0.149995,0.186131,0.179794
O3 Mean,-0.525557,-0.392298,1.000000,-0.170427,-0.336845,-0.160823
SO2 Mean,0.416436,0.149995,-0.170427,1.000000,0.275931,-0.114577
population_density,0.437043,0.186131,-0.336845,0.275931,1.000000,0.047009
num_factory,0.294672,0.179794,-0.160823,-0.114577,0.047009,1.000000


In [32]:
df8p.corr()

,NO2 Mean,CO Mean,O3 Mean,SO2 Mean,population_density,num_factory,Autos,Trucks,Total Vehicles
NO2 Mean,1.000000,0.511820,-0.105867,0.166962,0.257804,0.469287,0.594719,0.656627,0.613328
CO Mean,0.511820,1.000000,-0.288117,0.128114,-0.044839,0.102574,0.145071,0.141606,0.143774
O3 Mean,-0.105867,-0.288117,1.000000,-0.003403,-0.359067,-0.080305,-0.118654,-0.018385,-0.095352
SO2 Mean,0.166962,0.128114,-0.003403,1.000000,0.019531,-0.139716,-0.004078,0.042580,0.006573
population_density,0.257804,-0.044839,-0.359067,0.019531,1.000000,0.502800,0.525918,0.437381,0.506228
num_factory,0.469287,0.102574,-0.080305,-0.139716,0.502800,1.000000,0.905554,0.848553,0.894844
Autos,0.594719,0.145071,-0.118654,-0.004078,0.525918,0.905554,1.000000,0.978361,0.998608
Trucks,0.656627,0.141606,-0.018385,0.042580,0.437381,0.848553,0.978361,1.000000,0.987782
Total Vehicles,0.613328,0.143774,-0.095352,0.006573,0.506228,0.894844,0.998608,0.987782,1.000000


In [33]:
## in cells below, change var to use each of 4 pollutants

In [34]:
pollutants = ['NO2 Mean','CO Mean','O3 Mean','SO2 Mean']
num_trials = 50
mse_list = []
mae_list = []
scr_list = []
for pollutant in pollutants:
    for i in range(num_trials):
        train7,test7 = train_test_split(df7p, test_size=0.2, random_state=i+1)
        X7_train = train7.drop([pollutant], axis=1)
        y7_train = train7[pollutant]
        X7_test = test7.drop([pollutant], axis=1)
        y7_test = test7[pollutant]
        
        p_model = LinearRegression()
        p_model.fit(X7_train, y7_train)
    
        predicted_no2 = p_model.predict(X7_test)
        mse_list.append(mean_squared_error(y7_test, predicted_no2))    
        mae_list.append(mean_absolute_error(y7_test, predicted_no2))
        scr_list.append(p_model.score(X7_test,y7_test))
           
    print('linear regression without vehicles - ',num_trials,' random trials', '(',pollutant,')')
    print('R square:', np.average(scr_list))
    print('mse:', np.average(mse_list))
    print('mae:', np.average(mae_list))


linear regression without vehicles -  50  random trials ( NO2 Mean )
R square: 0.6049526630493204
mse: 15.040630004163532
mae: 3.0147511400718168
linear regression without vehicles -  50  random trials ( CO Mean )
R square: 0.4835323995373891
mse: 7.5311550617482474
mae: 1.561371665141591
linear regression without vehicles -  50  random trials ( O3 Mean )
R square: 0.42257256822066247
mse: 5.020775998975639
mae: 1.0420210033530732
linear regression without vehicles -  50  random trials ( SO2 Mean )
R square: 0.37863895012339427
mse: 4.551442621208937
mae: 1.1173598901434554
